In [1]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [2]:
df = pd.read_json(r'D:\HUST\2023.2\Machine Learning\Project\Data\preprocessed_movieDB.json')

,title,duration,user_score,number_of_vote,budget,revenue,restriction,year,month,star_power_actors,...,genre_history,genre_horror,genre_music,genre_mystery,genre_romance,genre_science fiction,genre_thriller,genre_tv movie,genre_war,genre_western
0,Four Rooms,98,58,2560,4000000,4257354,5,1995,12,9.098045e+08,...,0,0,0,0,0,0,0,0,0,0
1,Judgment Night,109,65,321,21000000,12136938,5,1993,10,5.573047e+08,...,0,0,0,0,0,0,1,0,0,0
2,Star Wars,121,82,19970,11000000,775398007,3,1977,5,1.336120e+09,...,0,0,0,0,0,1,0,0,0,0
3,Finding Nemo,100,78,18701,94000000,940335536,0,2003,5,3.378325e+09,...,0,0,0,0,0,0,0,0,0,0
4,Forrest Gump,142,85,26574,55000000,677387716,4,1994,7,9.892606e+08,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6229,Climax,97,71,1992,2900000,1700000,5,2018,9,1.694463e+07,...,0,1,0,0,0,0,0,0,0,0
6230,Jurassic World Dominion,147,67,5769,165000000,1001978080,4,2022,6,1.420629e+09,...,0,0,0,0,0,1,0,0,0,0
6231,Five Nights at Freddy's,110,76,3628,20000000,294803785,4,2023,10,3.598118e+08,...,0,1,0,1,0,0,0,0,0,0
6232,Alpviram,140,80,1,70000,75000,2,2009,5,7.500000e+04,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df.drop(columns = 'title', axis = 1, inplace = True)

In [4]:
import numpy as np
df['log_revenue'] = np.log(df['revenue'] + 1)
df['log_budget'] = np.log(df['budget'] + 1)
df.drop(['revenue','budget'], axis=1, inplace=True)
df

,duration,user_score,number_of_vote,restriction,year,month,star_power_actors,star_power_directors,genre_action,genre_adventure,...,genre_music,genre_mystery,genre_romance,genre_science fiction,genre_thriller,genre_tv movie,genre_war,genre_western,log_revenue,log_budget
0,98,58,2560,5,1995,12,9.098045e+08,5.001889e+08,0,0,...,0,0,0,0,0,0,0,0,15.264159,15.201805
1,109,65,321,5,1993,10,5.573047e+08,3.604714e+08,1,0,...,0,0,0,0,1,0,0,0,16.311764,16.860033
2,121,82,19970,3,1977,5,1.336120e+09,3.341551e+09,1,1,...,0,0,0,1,0,0,0,0,20.468887,16.213406
3,100,78,18701,0,2003,5,3.378325e+09,1.745747e+09,0,0,...,0,0,0,0,0,0,0,0,20.661747,18.358805
4,142,85,26574,4,1994,7,9.892606e+08,3.613586e+09,0,0,...,0,0,1,0,0,0,0,0,20.333754,17.822844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6229,97,71,1992,5,2018,9,1.694463e+07,8.954249e+06,0,0,...,0,0,0,0,0,0,0,0,14.346139,14.880222
6230,147,67,5769,4,2022,6,1.420629e+09,1.001978e+09,1,1,...,0,0,0,1,0,0,0,0,20.725242,18.921456
6231,110,76,3628,4,2023,10,3.598118e+08,2.948038e+08,0,0,...,0,1,0,0,0,0,0,0,19.501821,16.811243
6232,140,80,1,2,2009,5,7.500000e+04,1.480475e+09,0,0,...,0,0,0,0,0,0,0,0,11.225257,11.156265


In [5]:
X = df.drop('log_revenue', axis=1)
y = df['log_revenue']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
model = XGBRegressor(objective='reg:squarederror')
model.fit(X_train_scaled, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [9]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# Define parameter distributions for randomized search
param_dist = {
    'learning_rate': uniform(0.05, 0.1),  # Adjusted learning rates
    'n_estimators': randint(200, 500),  # Adjusted number of estimators
    'max_depth': randint(3, 7),  # Adjusted maximum depth of trees
    'min_child_weight': randint(1, 4),  # Adjusted minimum child weight
    'subsample': uniform(0.8, 0.2),  # Adjusted subsample ratio
    'colsample_bytree': uniform(0.6, 0.4),  # Adjusted column subsample ratio
    'gamma': uniform(0, 0.2),  # Adjusted gamma parameter
    'reg_alpha': uniform(0, 0.005),  # Adjusted L1 regularization term
    'reg_lambda': uniform(0, 0.005)  # Adjusted L2 regularization term
}

cv = KFold(n_splits=5, shuffle=True, random_state=42)
# Initialize the RandomizedSearchCV object
random_search = RandomizedSearchCV(
    estimator=model, 
    param_distributions=param_dist, 
    n_iter=150,  # Adjusted number of iterations
    scoring=['r2', 'neg_mean_squared_error'], 
    cv=cv, 
    refit='neg_mean_squared_error', 
    verbose=2
)

# Perform the randomized search
random_search.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 150 candidates, totalling 750 fits
[CV] END colsample_bytree=0.7496807397892772, gamma=0.1503347628689243, learning_rate=0.08313081996406213, max_depth=5, min_child_weight=3, n_estimators=405, reg_alpha=0.001975613525186542, reg_lambda=0.004810092738741553, subsample=0.997811649911022; total time=   0.2s
[CV] END colsample_bytree=0.7496807397892772, gamma=0.1503347628689243, learning_rate=0.08313081996406213, max_depth=5, min_child_weight=3, n_estimators=405, reg_alpha=0.001975613525186542, reg_lambda=0.004810092738741553, subsample=0.997811649911022; total time=   0.1s
[CV] END colsample_bytree=0.7496807397892772, gamma=0.1503347628689243, learning_rate=0.08313081996406213, max_depth=5, min_child_weight=3, n_estimators=405, reg_alpha=0.001975613525186542, reg_lambda=0.004810092738741553, subsample=0.997811649911022; total time=   0.1s
[CV] END colsample_bytree=0.7496807397892772, gamma=0.1503347628689243, learning_rate=0.08313081996406213, max_depth=5, min_

In [ ]:
# Get the best model from the randomized search
best_model_random = random_search.best_estimator_

# Evaluate the best model on the test set
y_pred_random = best_model_random.predict(X_test_scaled)
mse_random = mean_squared_error(y_test, y_pred_random)
rmse_random = mse_random ** 0.5
mae_random = median_absolute_error(y_test, y_pred_random)
r2_random = r2_score(y_test, y_pred_random)

# Print the results
print(f'Best Parameters: {random_search.best_params_}')
print(f'MAE: {mae_random}')
print(f'RMSE: {rmse_random}')
print(f'R²: {r2_random}')

Best Parameters: {'colsample_bytree': 0.8308332992928857, 'gamma': 0.19445463427154797, 'learning_rate': 0.06381269839806342, 'max_depth': 4, 'min_child_weight': 3, 'n_estimators': 452, 'reg_alpha': 0.00062794562851615, 'reg_lambda': 0.0012774318616169268, 'subsample': 0.8129663746306511}
MAE: 0.4512237715139733
RMSE: 1.1341489368495723
R²: 0.8119432065764296
